## TensorFlow 实现自定义操作 -- Dense

利用 TF 提供的 C++ API 我们可以实现一个自定义的操作。在这里我们实现一个类似于 NN 中的 DenseLayer 操作，其支持 FP 和 BP 的梯度计算，只是没有 Bias.

- [ ] Bias 实现。
- [ ] FM, FFM 层实现。

能够用 C++ 实现 Layers 是基础，因为我们可以需要自己实现 FM, FFM 层。 

### 目的

我们先看下 NN 中 Dense Layer 的操作。在 Dense Layer 中我们需要一个权重矩阵 $W^{(l)} \in R^{n_l \times n_{l - 1}}$ 和一个训练样本 $x, x \in R^{n_{l - 1} \times 1}$

即有 FP 操作：

$$
z^{(l)} = W^{(l)} a^{(l - 1)} \\
a^{(l)} = \sigma(z^{(l)})
$$

假设一个很简单的模型：

$$
y = Wx
$$

我们以第一层为例。假设  $a^{(1)} = x^{(1)} = [x_1, x_2, x_3]^T$, $ W^{(2)}$ 为：

$$
W^{(2)} = 
 \begin{pmatrix}
  w_{1,1} & w_{1,2} & w_{1,3} \\
  w_{2,1} & w_{2,2} & w_{2,3} 
 \end{pmatrix}
$$

则有 $z^{(2)}$

$$
z^{(2)} = W^{(2)} a^{(1)} = 
 \begin{pmatrix}
  w_{1,1} & w_{1,2} & w_{1,3} \\
  w_{2,1} & w_{2,2} & w_{2,3} 
 \end{pmatrix}
  \begin{pmatrix}
  x_{1} \\
  x_{2} \\
  x_{3}
 \end{pmatrix} =  \begin{pmatrix}
  w_{1,1}x_{1} + w_{1,2}x_{2} + w_{1,3} x_{3}\\
  w_{2,1}x_{1} + w_{2,2}x_{2} + w_{2,3} x_{3}
 \end{pmatrix} = 
   \begin{pmatrix}
  \hat{y_{1}} \\
  \hat{y_{2}}
 \end{pmatrix}
$$



则有 loss 函数 

$$
L = (\hat{y} - y)^2
$$

根据自动微分 AD 的求导法则，有


$$
\frac{dL}{dw_{i,j}} = \sum_{k=1}^2 \frac{dL}{d{\hat{y_k}}} \frac{d{\hat{y_k}}}{dw_{i,j}} = 2 * (\hat{y}_i - y_i) * \frac{d{\hat{y_k}}}{dw_{i,j}} = \bigg(2 * (\hat{y}_i - y_i)\bigg) * x_i
$$


$w_{i,j}$ 仅与一个 $y_i$ 相关，其它的为 0；而下面的 $x_{i}$ 则与所有的 $y_i$ 相关。


$$
\frac{dL}{dx_{i}} = \sum_{k=1}^2 \frac{dL}{d{\hat{y_k}}} \frac{d{\hat{y_k}}}{dx_{i}} = \sum_{k=1}^2 2 * (\hat{y}_i - y_i) * \frac{d{\hat{y_k}}}{dx_{i}} = \bigg(2 * (\hat{y}_1 - y_1)\bigg) * w_{1, i} + \bigg(2 * (\hat{y}_2 - y_2)\bigg) * w_{2, i}
$$

其中 $\bigg(\bigg)$ 括起来的是逆向上一个结点的梯度，在实现时我把上个结点的梯度考虑进来。

具体的实现参考: [InnerProduct](https://github.com/AaronFlower/cplusplus/tree/master/21-tensorflow/02-inner-product-op)